# Partie 1. - Pré-traitements du texte

Dans cette partie, nous allons étudier les principes de base du pré-traitement de textes.

L'objectif du pré-traitement est de :
- normaliser le texte : convertir tout en minuscule, ramener les mots à leur forme canonique,  ...
- découper le texte en _tokens_ au niveau des espaces, des signes de ponctuation, ...
- représenter le texte selon un formalisme mathématique pour permettre son analyse
- finalement : préparer le texte à la mise en oeuvre de la tâche de TALN à accomplir.

Dans cette partie, nous allons mettre en oeuvre quelques techniques de pré-traitements du texte et visualiser leurs intérêts. Pour cela, nous allons nous appuyer sur la bibliothèque de TALN _Spacy_ (https://spacy.io/) qui offre de nombreuses fonctionnalités pour la mise en place de traitement du langage en milieu industriel. 

## Import des bibliothèques logicielles et configuration

In [19]:
import spacy # Bibliothèque de TALN
from spacy.lang.en.stop_words import STOP_WORDS # liste des mots vides en anglais
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop # liste des mots vides en français
from spacy import displacy # visualisation des résultats
import pandas as pd # manipulation de données
import re # mise en oeuvre d'expression régulières
import plotly.express as px # création de graphiques et visuels
import ast # à ignorer pour ce TP

# Téléchargement de modèles entraînés de TALN pour l'anglais et le français. *Attention* : nécessite Internet et peut prendre qques minutes.
#!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_md

/home/vincent/dev/FORMATION_DATA_SCIENCES/taln-course/venv/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
You should consider upgrading via the '/home/vincent/dev/FORMATION_DATA_SCIENCES/taln-course/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [20]:
# Configuration

# Chargement du modèle de TALN Français.
nlp = spacy.load('fr_core_news_md')
nlp.max_length = 400000000

## 1. Tokenisation

La tokenisation consiste à découper une phrase en mots ou _tokens_. Le découpage se fait généralement au niveau des espaces, des signes de ponctuation et des retours à la ligne. 

Avant d'exploiter le jeu de données créé précédemment, nous allons travailler sur le texte de taille réduite ci-dessous (extrait de Wikipedia) afin de comprendre les algorithmes mis en oeuvre.

In [21]:
text = '''
Naval Group est un groupe industriel français spécialisé dans la construction navale de défense. 
Le groupe emploie 15 792 personnes en 2020 à travers dix-huit pays. 
Société de droit privé détenue principalement à hauteur de 62,49 % par l’État français et de 35 % par Thales, 
Naval Group est, depuis 2017, l’héritière des arsenaux français et de la Direction des constructions et armes navales (DCAN), 
devenue la Direction des constructions navales (DCN) en 1991 et DCNS en 2007 (le « S » ajouté pour la notion de système et de service). 
Depuis 2021, le groupe se recentre sur ses activités navales. 
'''

Le code ci-dessous est une fonction permettant d'afficher les tokens les plus fréquent dans une liste de tokens. On l'utilisera par la suite pour visualiser le résultat de différentes tokenisation.

In [22]:
def show_most_frequent_tokens(tokens, max=50):
    """ Fonction permettant d'afficher une liste de tokens sous la forme d'un graphique barre."""
    uniq_tokens = set(tokens)
    token_count = {}
    for token in uniq_tokens:
        count = tokens.count(token)
        token_count[token] = count
    s = pd.Series(token_count).sort_values(ascending=False).head(max)
    fig = px.bar(s)
    fig.show()

### 1.1. Tokenisation simple
La fonction ci-dessous permet de tokeniser un texte de manière (un peu trop) simple.

In [23]:
def tokenize(nlp, text):
    """Fonction retournant la liste des tokens dans le texte `text` passé en paramètre"""
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

**Exercice** : appliquez la fonction de tokenisation au texte (variable `text`) et affichez le résultat. Le premier paramètre de la fonction est l'objet `nlp` défini plus haut.

In [24]:
tokens = [] # TODO : remplacer le tableau vide [] l'appel de la fonction tokenize().
print(tokens)

[]


**Exercice** : en utilisant la fonction `show_most_frequent_tokens`, afficher la liste des tokens les plus fréquent dans le texte.

In [ ]:
show_most_frequent_tokens([]) # TODO

La tokenisation a bien été réalisée mais elle comporte plusieurs écueils :
- les signes de ponctuation sont considérés comme des tokens ;
- les caractères spéciaux (`\n` : caractère de retour à la ligne) sont aussi considérés comme des tokens ;
- les mots ne sont pas normalisés (majuscules/minuscules ; pluriel/singulier ; ...)

Améliorons notre tokenizer.

### 1.2. Tokenisation avec normalisation des tokens

In [ ]:
punctuations = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
special_chars = ['\n', '\r', '»', '«']

forbidden_words = punctuations + special_chars + list(fr_stop)

def tokenize_with_normalization(nlp, text):
    """Fonction retournant la liste des tokens normalisés dans le texte `text` passé en paramètre."""
    text_normalized = text.lower() # conversion du texte en minuscules.
    doc = nlp(text_normalized)
    tokens = [token.text for token in doc if token.text not in forbidden_words] # suppression des caractères spéciaux et des mots vides.
    return tokens

**Exercice** : appliquez la fonction de tokenisation avec normalisation au texte `text` et affichez le résultat.

In [ ]:
tokens_normalized = [] # TODO
print(tokens)

**Exercice** : en utilisant la fonction `show_most_frequent_tokens`, afficher la liste des tokens les plus fréquent dans le texte.

In [ ]:
show_most_frequent_tokens([])

Le résultat est plus cohérent mais il faudrait maintenant regrouper les termes ayant le même sens. Par exemple `navale` et `navales` véhiculent le même concept, ils devraient donc être associés au même token.

Pour cela nous allons utiliser la _lemmatisation_, principe qui consiste à calculer le lemme d'un mot, c'est à dire une forme canonique. Pour un nom par exemple, il s'agit de sa forme au masculin singulier.

### 1.3. Normalisation par lemmatisation

In [ ]:
punctuations = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
special_chars = ['\n', '\r', '»', '«']

forbidden_words = punctuations + special_chars + list(fr_stop)

def tokenize_with_lemmatization(nlp, text):
    """Fonction retournant la liste des tokens lemmatisés dans le texte `text` passé en paramètre."""
    text = re.sub('\n','',text).lower()
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in forbidden_words and len(token.lemma_) > 1]
    return tokens

**Exercice** : appliquez la fonction de tokenisation avec lemmatisation au texte (variable `text`) et affichez le résultat.

In [ ]:
tokens_lemmatized = tokenize_with_lemmatization(nlp, text) # TODO
print(tokens_lemmatized)

**Exercice** : en utilisant la fonction `show_most_frequent_tokens`, afficher la liste des tokens les plus fréquents dans le texte.

In [ ]:
show_most_frequent_tokens(tokens_lemmatized) # TODO

Le résultat est maintenant plus cohérent : 
- les termes les plus fréquents sont _naval_, _construction_ : logique ;
- les mots partageant un même lemme ont été regroupés ensemble.

Il reste tout de même quelques imperfections : 
- on aurait aimé que les mots composés soient dans un même token. Par exemple `Naval Group` ne doit pas être découpé ;

Des techniques existent pour réduire les imperfections de l'approche : l'extraction des entités nommées que nous verrons en Partie 3 ou l'analyse morpho-syntaxique que nous allons voir maintenant.

## 2. Analyse morpho-syntaxique

L'analyse morphosyntaxique consiste à catégoriser les mots en leur affectant une étiquette indiquant si le mot est un `nom`, un `verbe`, un `adjectif` etc. Cette technique permet de mieux comprendre le texte, d'interpréter une partie du sens des mots, de trouver les relations entre les mots et / ou de focaliser l'analyse uniquement sur une catégorie particulière de mots. 

In [ ]:
def tokenize_with_morpho_syntax_analysis(nlp, text, render=False):
    """Fonction retournant la liste des tokens et leur étiquette grammaticale."""
    doc = nlp(text)
    tokens = [(token.text, token.pos_) for token in doc if token.lemma_ not in forbidden_words]
    if render:
        displacy.render(doc, style="dep")
    return tokens

**Exercice** : appliquez la fonction de tokenisation avec analayse morpho syntaxique au texte (variable `text`) et affichez le résultat. La variable booléenne `render` permet d'afficher ou non le graphe de dépendances.

In [ ]:
tokens_morpho_syntax = [] # TODO
print(tokens_morpho_syntax)

Maintenant, créons une fonction de tokenisation pour ne garder que les noms propres `PROPN`.

In [ ]:
def tokenize_with_morpho_syntax_analysis_propn(nlp, text):

    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in forbidden_words and  token.pos_ == 'PROPN']
    return tokens

**Exercice** : modifier le tokenizer ci-dessous pour ne conserver que le noms propres : `token.pos_ == 'PROPN'`

In [ ]:
tokens_proper_nouns = [] # TODO
show_most_frequent_tokens(tokens_proper_nouns)

Avec ces techniques, nous commençons à parvenir à extraire des informations intéressantes du texte !

## 2. Application au jeu de données pour trouver les mots les plus fréquents

Pour tester notre pré-traitement, nous allons charger le jeu de données vue en première partie et y appliquer la tokenisation.

Pour simplifier la tâche, nous allons fusionner l'ensemble des documents pour trouver les termes les plus fréquents.

In [ ]:
# Chargement du jeu de données enregistré à la partie 1.
f = open("./dataset_processed/dataset.json", "r")
documents_raw = f.read()
documents = ast.literal_eval(documents_raw)
f.close()

In [ ]:
# Concaténation des documents pour réaliser l'analyse.
big_text = ' '.join([doc['content'] for doc in documents.values()])
big_text = big_text[0:100000]

**Exercice** : appliquez la fonction la fonction de tokenisation au texte `big_text` de votre choix puis afficher les tokens les plus fréquents à l'aide de la fonction `show_most_frequent_tokens`.

In [ ]:
tokens = [] # TODO
show_most_frequent_tokens(tokens)

## Conclusion

Dans cette partie , nous avons comment découper le textes en Token et l'importance de la normalisation pour obtenir une liste cohérente de tokens.